In [4]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic
from anthropic.types import Message


load_dotenv()

client = Anthropic()
model = "claude-3-7-sonnet-latest"
num_tokens = 2000
think_cost = 1024

In [5]:
# Helper functions

# Magic string to trigger redacted thinking
thinking_test_str = "ANTHROPIC_MAGIC_STRING_TRIGGER_REDACTED_THINKING_46C9A13E193C177646C7398A98432ECCCE4C1253D5E2D82641AC0E52CC2876CB"


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=think_cost
):
    params = {
        "model": model,
        "max_tokens": num_tokens,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system
   
    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget": thinking_budget
        }

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [8]:
messages = []

add_user_message(messages, "What is saltpeter?")

chat(messages)

Message(id='msg_017GvV6Q7ZLpXgyhrumfVxdq', content=[TextBlock(citations=None, text='Saltpeter is the common name for potassium nitrate (KNO₃), a naturally occurring crystalline salt that has been historically important for several uses:\n\n- It was a key ingredient in early gunpowder (mixed with sulfur and charcoal)\n- Used in food preservation as a curing salt for meats\n- Applied as a fertilizer due to its high nitrogen content\n- Utilized in various industrial processes and in medicine\n\nHistorically, saltpeter was collected from cave deposits or manufactured in "nitre beds" where organic materials were allowed to decompose. Today, it\'s primarily produced through chemical processes.', type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=13, output_tokens=139, server_tool_use=None, service_tier='standard', cache_creation={'ep